This document is a review of https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

# Titanic

In [2]:
import seaborn as sns
titanic = sns.load_dataset('titanic')

In [3]:
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


## groupby() and unstack()

In [4]:
titanic.groupby(['sex','class'])[['survived']].agg(['mean']).index.get_level_values('class')

CategoricalIndex([u'First', u'Second', u'Third', u'First', u'Second',
                  u'Third'],
                 categories=[u'First', u'Second', u'Third'], ordered=False, name=u'class', dtype='category')

In [5]:
titanic.groupby(['sex','class'])[['survived']].agg(['mean']).reset_index()

,sex,class,survived
,,,mean
0,female,First,0.968085
1,female,Second,0.921053
2,female,Third,0.500000
3,male,First,0.368852
4,male,Second,0.157407
5,male,Third,0.135447


In [6]:
titanic.groupby(['sex','class'])[['survived']].agg(['mean'])

survived
                   mean
sex    class           
female First   0.968085
       Second  0.921053
       Third   0.500000
male   First   0.368852
       Second  0.157407
       Third   0.135447

In [7]:
titanic.groupby(['sex','class'])[['survived']].agg(['mean']).unstack()

survived                    
            mean                    
class      First    Second     Third
sex                                 
female  0.968085  0.921053  0.500000
male    0.368852  0.157407  0.135447

## pivot_table()

In [8]:
titanic.pivot_table(index='sex',columns='class',values='survived') # default agg function is np.mean()

class,First,Second,Third
sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


In [9]:
age = pd.cut(titanic['age'],[0,18,80])
titanic.pivot_table(index=['sex',age],columns='class',values='survived')

class               First    Second     Third
sex    age                                   
female (0, 18]   0.909091  1.000000  0.511628
       (18, 80]  0.972973  0.900000  0.423729
male   (0, 18]   0.800000  0.600000  0.215686
       (18, 80]  0.375000  0.071429  0.133663

In [10]:
fare = pd.qcut(titanic['fare'],2)
titanic.pivot_table('survived',['sex',age],[fare,'class'])

fare            [0, 14.454]                     (14.454, 512.329]            \
class                 First    Second     Third             First    Second   
sex    age                                                                    
female (0, 18]          NaN  1.000000  0.714286          0.909091  1.000000   
       (18, 80]         NaN  0.880000  0.444444          0.972973  0.914286   
male   (0, 18]          NaN  0.000000  0.260870          0.800000  0.818182   
       (18, 80]         0.0  0.098039  0.125000          0.391304  0.030303   

fare                       
class               Third  
sex    age                 
female (0, 18]   0.318182  
       (18, 80]  0.391304  
male   (0, 18]   0.178571  
       (18, 80]  0.192308

In [11]:
## relationship between class and fare??
titanic.box()

AttributeError: 'DataFrame' object has no attribute 'box'

In [ ]:
titanic.pivot_table(index='sex',columns='class',aggfunc={'survived':'sum','fare':'mean'})
# sum and np.mean without quotation marks works because sum is a python function but mean is not

In [ ]:
titanic.pivot_table('survived',index='sex',columns='class',margins=True)

# Birth

In [ ]:
births = pd.read_csv('data/births.csv')

In [ ]:
births.head()

In [ ]:
births['decade'] = 10*(births['year']//10)
decade_gender = births.pivot_table(index='decade',columns=['gender'],aggfunc={'births':'sum'})

In [ ]:
decade_gender

In [ ]:
# sns.set_style("whitegrid")
fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(111)
ax.plot(decade_gender)

In [ ]:
fig = plt.figure(figsize=(8,5))
ax1 = fig.add_subplot(111)
births.pivot_table(values='births',index='year',columns='gender',aggfunc='sum').plot(ax=ax1)
plt.ylabel('total births per year')

## sigma clipping operation

In [ ]:
quartiles = np.percentile(births['births'],[25,50,75])
mu = quartiles[1]
# 0.74 is interquantile range of a gaussian distribution
sig = 0.74*(quartiles[2]-quartiles[0])

births = births.query('(births > @mu-5*@sig) & (births < @mu+5*@sig)')

In [ ]:
births['day'] = births['day'].astype(int)

In [ ]:
# create a datetime index from the year, month, day
births.index = pd.to_datetime(10000 * births.year +
                              100 * births.month +
                              births.day, format='%Y%m%d')

births['dayofweek'] = births.index.dayofweek

In [ ]:
decade_birth = births.pivot_table('births', index='dayofweek',columns='decade');decade_birth

In [ ]:
decade_birth.plot()
plt.gca().set_xticklabels(['Mon', 'Tues', 'Wed', 'Thurs', 'Fri', 'Sat', 'Sun'])
plt.ylabel('mean births by day')

Realize that 1990 and 2000 do not appear in the plot because their records are dropped using sigma clipping operation. 1990 and 2000 decades do not have daily record; instead, their births are monthly aggregated results.

In [ ]:
births_by_date = births.pivot_table(values='births',index=[births.index.month, births.index.day])
births_by_date.head()

In [ ]:
births_by_date.index = [pd.datetime(2012,month,day) for (month,day) in births_by_date.index]
births_by_date.head()

In [ ]:

fig, ax = plt.subplots(figsize=(10,5))
births_by_date.plot(ax=ax)
ax.annotate("New Year's Day", xy=('2012-1-1', 4100),  xycoords='data',
            xytext=(50, -30), textcoords='offset points',
            arrowprops=dict(arrowstyle="->",
                            connectionstyle="arc3,rad=-0.2"))

ax.annotate("Independence Day", xy=('2012-7-4', 4250),  xycoords='data',
            bbox=dict(boxstyle="round", fc="none", ec="gray"),
            xytext=(10, -40), textcoords='offset points', ha='center',
            arrowprops=dict(arrowstyle="->"))

ax.annotate('Labor Day', xy=('2012-9-4', 4850), xycoords='data', ha='center',
            xytext=(0, -20), textcoords='offset points')
ax.annotate('', xy=('2012-9-1', 4850), xytext=('2012-9-7', 4850),
            xycoords='data', textcoords='data',
            arrowprops={'arrowstyle': '|-|,widthA=0.2,widthB=0.2', })

ax.annotate('Halloween', xy=('2012-10-31', 4600),  xycoords='data',
            xytext=(-80, -40), textcoords='offset points',
            arrowprops=dict(arrowstyle="fancy",
                            fc="0.6", ec="none",
                            connectionstyle="angle3,angleA=0,angleB=-90"))

ax.annotate('Thanksgiving', xy=('2012-11-25', 4500),  xycoords='data',
            xytext=(-120, -60), textcoords='offset points',
            bbox=dict(boxstyle="round4,pad=.5", fc="0.9"),
            arrowprops=dict(arrowstyle="->",
                            connectionstyle="angle,angleA=0,angleB=80,rad=20"))


ax.annotate('Christmas', xy=('2012-12-25', 3850),  xycoords='data',
             xytext=(-30, 0), textcoords='offset points',
             size=13, ha='right', va="center",
             bbox=dict(boxstyle="round", alpha=0.1),
             arrowprops=dict(arrowstyle="wedge,tail_width=0.5", alpha=0.1));

# Label the axes
ax.set(title='USA births by day of year (1969-1988)',
       ylabel='average daily births')

# Format the x axis with centered month labels
ax.xaxis.set_major_locator(mpl.dates.MonthLocator())
ax.xaxis.set_minor_locator(mpl.dates.MonthLocator(bymonthday=15))
ax.xaxis.set_major_formatter(plt.NullFormatter())
ax.xaxis.set_minor_formatter(mpl.dates.DateFormatter('%h'));

ax.set_ylim(3600, 5400);